In [1]:
import h5py
import pandas as pd
import numpy as np
import os

In [2]:
!pwd

/home/jovyan/workspace/Desktop/bsc_outputs/particules_solitaries


In [3]:
def populate_tables(data_folder_name='./data_p', 
                   tracers='tip'):
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'{tracers}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    while time < num_files:
        print(f'time: {time}')
        ## COORDINATES ##
        # open Coordinates H5 
        h5_file_coord =  f'./{data_folder_name}/{tracers}-{time}.h5'
        if not os.path.exists(h5_file_coord):
                print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name to store coord
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
                
        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #print(df_name_by_t)

        ## Velocity Gradient TENSOR ##
        # open the Strain Rate Tensor H5 files 
        h5_file_path =  f'./{data_folder_name}/{prefix}-{time}.h5'
        if not os.path.exists(h5_file_path):
                print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_path, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
            
        # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
        #globals()[dataset_name][0]
        globals()[df_name_by_t]['VG00'] = globals()[dataset_name][:, 0]
        globals()[df_name_by_t]['VG01'] = globals()[dataset_name][:, 1]
        globals()[df_name_by_t]['VG02'] = globals()[dataset_name][:, 2]
        globals()[df_name_by_t]['VG10'] = globals()[dataset_name][:, 3]
        globals()[df_name_by_t]['VG11'] = globals()[dataset_name][:, 4]
        globals()[df_name_by_t]['VG12'] = globals()[dataset_name][:, 5]
        globals()[df_name_by_t]['VG20'] = globals()[dataset_name][:, 6]
        globals()[df_name_by_t]['VG21'] = globals()[dataset_name][:, 7]
        globals()[df_name_by_t]['VG22'] = globals()[dataset_name][:, 8]
          
        ## save the df as txt
        output_folder = 'txt_outputs_test'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')
            
        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)
            
        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########
            
            
        print(f'hi {df_name_by_t}') 
        print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [4]:
populate_tables()

17
time: 0
out folder:  txt_outputs_test
hi df_t_0
   index            x            y      z  VG00  VG01  VG02  VG10  VG11  VG12  \
0      0  1600.000000   455.555556 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   
1      1  1733.333333   922.222222 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   
2      2  1600.000000  1544.444444 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   

   VG20  VG21  VG22  
0   0.0   0.0   0.0  
1   0.0   0.0   0.0  
2   0.0   0.0   0.0  
time: 1
hi df_t_1
   index            x            y           z          VG00          VG01  \
0      0  1600.128476   454.980801 -401.035047  7.259619e-17  1.996495e-17   
1      1  1733.583287   921.934147 -400.593724 -7.111254e-17  1.325494e-17   
2      2  1600.521556  1544.466380 -400.918692 -5.055072e-19 -3.647725e-18   

           VG02          VG10          VG11          VG12          VG20  \
0  8.171243e-16 -2.740976e-17  2.029339e-17  1.051774e-16  3.704328e-16   
1  4.814289e-16  3.354163e-17 -1.330835e-17  9.666123e-

In [8]:
h5_file_coord =  f'./outputs_3d_no_t_copy/lag_velocitygradient-1.h5'

with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_6'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                 dataset_name = file[key][:] 
                    

dataset_name[0]

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 0:
  #000: H5F.c line 620 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3501 in H5VL_file_open(): failed to iterate over available VOL connector plugins
    major: Virtual Object Layer
    minor: Iteration failed
  #002: H5PLpath.c line 578 in H5PL__path_table_iterate(): can't iterate over plugins in plugin path '(null)'
    major: Plugin for dynamically loaded library
    minor: Iteration failed
  #003: H5PLpath.c line 620 in H5PL__path_table_iterate_process_path(): can't open directory: /usr/local/hdf5/lib/plugin
    major: Plugin for dynamically loaded library
    minor: Can't open directory or file
  #004: H5VLcallback.c line 3351 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_file.c line 97 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to o

RuntimeError: Unspecified error in H5Fopen (return value <0)